# データベース_ICSD_平均

In [1]:
# import module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)
/usr/lib/python2.7/dist-packages/PIL/Image.py:71: RuntimeWarning: The _imaging extension was built for another  version of Pillow or PIL
  warnings.warn(str(v), RuntimeWarning)


In [5]:
# make unnecessary compounds list
ld = open('/media/sf_kyoyu/data/complist_integer_compounds_pickup.csv')
lines = ld.readlines()
ld.close

delete_list = []
for i in ['He','Ne','Ar','Kr','Xe','Tc','Pm','Tb','Ne','Po','At','Rn','Fr','Ra','Ac','Th','Pa','U','Np','Pu','Am',\
          'Cm','Bk','Cf','Es','Fm','Md','No','Lr']:
    for line in lines:
        if line.find(i) >= 0:
            delete_list.append(line[:-1])
            
compounds = []
for j in delete_list:
    compounds.append(j.split()[0])
# print compounds

['He2Ne1', 'He1', 'He1', 'He1', 'N12Ne7', 'He2Ne1', 'Ne1', 'Ar1', 'Ar1', 'F20Kr5Sb2', 'F11Kr3Sb1', 'As2F16Kr3', 'F2Kr1', 'As1F7Kr1', 'Bi1F7Kr1', 'F7Kr1Sb1', 'F2Kr1', 'Kr1', 'Kr1', 'F8N2O2Xe1', 'As1F10N1S1Xe1', 'F2O2Xe1', 'F2O3S1Xe1', 'F1K1O3Xe1', 'Cr2F10Xe1', 'Cr1F10Xe1', 'B2Cd1F10Xe1', 'Cl1F36O9S1Sb1Te7Xe1', 'Cd1F22P2Xe5', 'F7O2Sb1Xe1', 'F15O1Sb1Xe2', 'H1F2N1O3Xe1', 'F2N2O4Xe1', 'F6O1W1Xe1', 'F9Sb1Xe1', 'As1F9O3S1Xe2', 'Bi1F9Xe1', 'F11Pt1Xe1', 'As1F12Li1Xe3', 'F18P2Sr1Xe3', 'F56P6Sr3Xe10', 'F58P6Pb3Xe11', 'Cu1F16Sb2Xe2', 'Cu1F20Sb2Xe4', 'C48H42Cl3O6Xe1', 'F14Sb2Xe1', 'Bi2F12Xe1', 'F5O4Tc1Xe1', 'Ba1F22Sb2Xe5', 'As2F16Mg1Xe2', 'F3N1O4S2Xe1', 'Cs1F7Xe1', 'F13N1O2Xe2', 'Au1F22Sb4Xe4', 'F10O2Te2Xe1', 'Au1F12Sb2Xe2', 'Au2F19Sb3Xe2', 'Au1F18Sb3Xe2', 'As2Ca1F20Xe4', 'As4Ca2F42Xe9', 'Cl1F36O9S1Sb1Te7Xe1', 'F24Sb2Xe6Zn1', 'Cu1F24Sb2Xe6', 'F6Xe1', 'F6Xe1', 'Ba1F22Ru2Xe5', 'Ba1F22Nb2Xe5', 'F36Ti8Xe2', 'F40Ti9Xe2', 'F16Sb2Xe1', 'Cs1F13O3Xe3', 'F18N1O4S2Sb3Xe1', 'F10Ge1Xe1', 'Au1F17Xe2', 'As1F9Xe2'

In [44]:
# 元データの整形
atomicdata=pd.read_csv('/media/sf_kyoyu/data/atomic_data (all_data).csv')
atomicdata=atomicdata.set_index('Symbol')
atomicdata=atomicdata.drop(atomicdata.ix['Po':'Rg'].index).drop(['RareEarth','Nval','IE2','IE3','Cp-g','Cp-mol',\
                                                                 'E-vapor','ElectricalConductivity','MolerMagneticSusceptibility',\
                                                                'ThermalExpansion'],axis=1).fillna(0)
dataframe = pd.DataFrame(atomicdata)
dataframe.to_csv('/media/sf_kyoyu/data/atomicdata処理後.csv')

complist_integer_compounds_pickup=pd.read_csv('/media/sf_kyoyu/data/complist_integer_compounds_pickup.csv',header=None,\
                                              delim_whitespace=True)
complist_integer_compounds_pickup.columns=["Compounds","H","He","Li","Be","B","C","N","O","F",\
                                           "Ne","Na","Mg","Al","Si","P","S","Cl","Ar","K","Ca","Sc",\
                                           "Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Ga","Ge","As",\
                                           "Se","Br","Kr","Rb","Sr","Y","Zr","Nb","Mo","Tc","Ru","Rh",\
                                           "Pd","Ag","Cd","In","Sn","Sb","Te","I","Xe","Cs","Ba","La",\
                                          "Ce","Pr","Nd","Pm","Sm","Eu","Gd","Tb","Dy","Ho","Er","Tm",\
                                          "Yb","Lu","Hf","Ta","W","Re","Os","Ir","Pt","Au","Hg","Tl","Pb","Bi"]
complist_integer_compounds_pickup = complist_integer_compounds_pickup.set_index("Compounds")

for i in compounds:
    complist_integer_compounds_pickup=complist_integer_compounds_pickup.drop(i)

complist_integer_compounds_pickup = complist_integer_compounds_pickup.reset_index().sort("Compounds")
complist_integer_compounds_pickup = \
complist_integer_compounds_pickup[~complist_integer_compounds_pickup["Compounds"].duplicated(take_last=False)]\
.set_index("Compounds").drop("index", axis=1)
df2 = pd.DataFrame(complist_integer_compounds_pickup)
df2.to_csv("/media/sf_kyoyu/data/complist_integer_compounds_pickup重複なし.csv")

,Compounds,H,He,Li,Be,B,C,N,O,F,Ne,Na,Mg,Al,Si,P,S,Cl,Ar,K,
40059,Ag1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
40023,Ag1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
25474,Ag1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...
296,Ag10O13Si4,0,0,0,0,0,0,0,13,0,0,0,0,0,4,0,0,0,0,0,...
4740,Ag11K1O16V4,0,0,0,0,0,0,0,16,0,0,0,0,0,0,0,0,0,0,1,...
5898,Ag13As3I4O12,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,...
5676,Ag15Cl3P4S16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,16,3,0,0,...
9358,Ag16Ca6N1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...
6876,Ag17Mg54,0,0,0,0,0,0,0,0,0,0,0,54,0,0,0,0,0,0,0,...
7259,Ag18Bi4O12,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,0,...


In [6]:
# make average database
A = complist_integer_compounds_pickup
B = atomicdata
C1 = A.dot(B)
for i in range(len(A.index)):
    C1.iloc[i , :] = C1.iloc[i , :]/(A.sum(axis=1)[i])

In [11]:
# average_databaseの重複の削除
C1=C1.reset_index().sort("Compounds")
C1=C1[~C1["Compounds"].duplicated(take_last=False)]
C1=C1.set_index("Compounds").drop("index", axis=1)
C1.to_csv("/media/sf_kyoyu/data/average_database_ICSD_重複なし.csv")
x = pd.read_csv("/media/sf_kyoyu/data/average_database_ICSD_重複なし.csv").set_index("Compounds").drop(['Z','Nf','CuKa','MoKa'], axis=1)

In [59]:
x = pd.read_csv("/media/sf_kyoyu/data/average_database_ICSD_重複なし.csv").set_index("Compounds").drop(['Z','Nf','CuKa','MoKa'], axis=1)\
.drop(['H1','Li1','Be1',"B1","C1","N1","O1","Na1","Mg1","Al1","Si1","P1","S1","K1","Ca1","Sc1",\
                                           "Ti1","V1","Cr1","Mn1","Fe1","Co1","Ni1","Cu1","Zn1","Ga1","Ge1","As1",\
                                           "Se1","Rb1","Sr1","Y1","Zr1","Nb1","Mo1","Ru1","Rh1",\
                                           "Pd1","Ag1","Cd1","In1","Sn1","Sb1","Te1","I1","Cs1","Ba1","La1",\
                                          "Ce1","Pr1","Nd1","Sm1","Eu1","Gd1","Dy1","Ho1","Er1","Tm1",\
                                          "Yb1","Lu1","Hf1","Ta1","W1","Re1","Os1","Ir1","Pt1","Au1","Hg1","Tl1","Pb1","Bi1"])

In [60]:
# 平均ー逆数データ
x_inverse = 1/x
x_inverse = x_inverse.replace('inf',0)

# 平均ー二乗データ
x_square = x*x

# 平均ー対数データ
x_log = np.log(x)
x_log = x_log.replace('-inf',0).fillna(0)

In [102]:
# 平均そのままデータの標準化
scaled_list = []
for i in np.array(x.columns):
    sc = StandardScaler()
    sc.fit(x.loc[:,i])
    array_scaled = sc.transform(x.loc[:,i])
    scaled_list.append(array_scaled)
df = pd.DataFrame(scaled_list).T
df.index = np.array(x.index)
df.columns = np.array(x.columns)

# 平均ー逆数データの標準化
scaled_list_inverse = []
for i in np.array(x_inverse.columns):
    sc = StandardScaler()
    sc.fit(x_inverse.loc[:,i])
    array_scaled_inverse = sc.transform(x_inverse.loc[:,i])
    scaled_list_inverse.append(array_scaled_inverse)
df_inverse = pd.DataFrame(scaled_list_inverse).T
df_inverse.index = np.array(x_inverse.index)
df_inverse.columns = np.array(x_inverse.columns)

# 平均ー二乗データの標準化
scaled_list_square = []
for i in np.array(x_square.columns):
    sc = StandardScaler()
    sc.fit(x_square.loc[:,i])
    array_scaled_square = sc.transform(x_square.loc[:,i])
    scaled_list_square.append(array_scaled_square)
df_square = pd.DataFrame(scaled_list_square).T
df_square.index = np.array(x_square.index)
df_square.columns = np.array(x_square.columns)

# 平均ー対数データの標準化
scaled_list_log = []
for i in np.array(x_log.columns):
    sc = StandardScaler()
    sc.fit(x_log.loc[:,i])
    array_scaled_log = sc.transform(x_log.loc[:,i])
    scaled_list_log.append(array_scaled_log)
df_log = pd.DataFrame(scaled_list_log).T
df_log.index = np.array(x_log.index)
df_log.columns = np.array(x_log.columns)

# 平均ー全データ連結
df_all = pd.concat([df,df_log], axis=1)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

In [135]:
# 主成分分析
# ------------平均そのままデータ
pca = PCA(n_components = 3, whiten = False)
pca.fit(df)
df_pca = pca.transform(df)
E = pca.explained_variance_ratio_
pca_components = pd.DataFrame(pca.components_ ,index = ["第一主成分","第二主成分","第三主成分"], columns = np.array(df.columns))
df_pca = pd.DataFrame(df_pca, columns = ["第一主成分","第二主成分","第三主成分"], index = df.index)
print pca.components_
print np.cumsum(E)

# ------------平均ALLデータ
pca = PCA(n_components = 3, whiten = False)
pca.fit(df_all)
df_pca_all = pca.transform(df_all)
E_all = pca.explained_variance_ratio_
pca_components_all = pd.DataFrame(pca.components_ ,index = ["第一主成分","第二主成分","第三主成分"], columns = np.array(df_all.columns))
df_pca_all = pd.DataFrame(df_pca_all, columns = ["第一主成分","第二主成分","第三主成分"], index = df_all.index)
# print pca.components_
# print np.cumsum(E_all)

[[ 0.29725097 -0.18926022 -0.02933847 -0.27885567  0.21975924  0.26852507
  -0.30671233 -0.17479003 -0.31517798  0.28735212  0.29488319  0.25195581
   0.29361414  0.27871964  0.15576434  0.17797388]
 [ 0.30691492  0.37125787  0.19323641  0.31798625  0.29827145  0.32968851
   0.10642406  0.36402275 -0.01042748  0.21311315  0.22157444 -0.21599951
  -0.13633192  0.17679401 -0.27311586 -0.14934689]
 [-0.03625584  0.3515558   0.36264495  0.15001532 -0.03020897 -0.02409552
   0.19890846  0.18486798  0.10711593 -0.04794625 -0.12929861  0.43644949
   0.3000202   0.12388066  0.56520504  0.0070656 ]]
[ 0.53922368  0.68098406  0.78018215]


In [116]:
df_pca_all[df_pca_all.ix[:,2] > 5]

,第一主成分,第二主成分,第三主成分
As1Cs3,6.641770,-1.409311,5.542368
Ba1Li4,3.057603,-7.476405,5.046507
Ba1Na2,5.287029,-5.884096,5.454478
Ba6N1Na16,4.877236,-6.294272,5.184316
Bi1Cs3,7.347399,-0.688215,6.500153
Bi1K3,5.333839,-2.553702,5.226084
Bi1Rb3,6.223238,-1.606614,5.847466
Bi2Cs3,6.590637,0.785483,5.243672
Cl1Cs3O1,1.582251,0.390870,5.189249
Cs11O3,4.408578,-1.550453,7.102535


In [137]:
# 主成分分析後の二次元プロット
# ----------------平均そのままデータ
plt.scatter(df_pca.ix[:,0], df_pca.ix[:,1])
plt.show()

# ----------------平均ALLデータ
plt.scatter(df_pca_all.ix[:,0], df_pca_all.ix[:,1])
plt.show()

In [131]:
# 三次元プロット
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter3D(df_pca_all.ix[:,0], df_pca_all.ix[:,1], df_pca_all.ix[:,2])
plt.show()

# 平均＋標準偏差の主成分分析

In [138]:
# 標準偏差データの読み込み
sd_data = pd.read_csv("/media/sf_kyoyu/data/standard_deviation_database_ICSD.csv").rename(columns = {'Unnamed: 0':'Compounds'})\
.set_index("Compounds")

# 標準化
scaled_list = []
for i in np.array(sd_data.columns):
    sc = StandardScaler()
    sc.fit(sd_data.loc[:,i])
    array_scaled = sc.transform(sd_data.loc[:,i])
    scaled_list.append(array_scaled)
df_sd = pd.DataFrame(scaled_list).T
df_sd.index = np.array(sd_data.index)
df_sd.columns = np.array(sd_data.columns)

# 平均＋標準偏差データ連結
df_average_sd = pd.concat([df, df_sd], axis=1)

# 主成分分析
pca = PCA(n_components = 3, whiten = False)
pca.fit(df_average_sd)
df_pca = pca.transform(df_average_sd)
E = pca.explained_variance_ratio_
pca_components = pd.DataFrame(pca.components_ ,index = ["第一主成分","第二主成分","第三主成分"], columns = np.array(df_average_sd.columns))
df_pca = pd.DataFrame(df_pca, columns = ["第一主成分","第二主成分","第三主成分"], index = df_average_sd.index)
print pca.components_
print np.cumsum(E)

# プロット
plt.scatter(df_pca.ix[:,0], df_pca.ix[:,1])
plt.show()

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.re

[[ 0.23918159 -0.14145978 -0.01912514 -0.23676024  0.20249143  0.21668012
  -0.25225745 -0.1517746  -0.25953076  0.21598057  0.22701104  0.21204698
   0.24877671  0.23964164  0.13185818  0.1594509  -0.11166961 -0.12619997
  -0.02544675 -0.21675534  0.11428625  0.02136789 -0.23321449 -0.13792662
  -0.23606154 -0.18959552 -0.17971982 -0.08399602 -0.13557921  0.06378911
  -0.04072257  0.03184389]
 [-0.22188292 -0.03754442  0.01922399 -0.10117008 -0.16771018 -0.21618665
   0.05259772 -0.15437373  0.08598408 -0.20750717 -0.23187853  0.26334879
   0.15638218 -0.06571953  0.36345602  0.06486986  0.01560766 -0.11384767
  -0.04528749 -0.13374426 -0.19618508 -0.06031673 -0.14545555 -0.19205338
  -0.10838369 -0.12815214 -0.09656937  0.33755334  0.18528008  0.010547
   0.42312472  0.02031014]
 [ 0.14591692 -0.00669182  0.09556531  0.00186349 -0.06195748  0.16931898
  -0.02021599  0.03287002 -0.05893611  0.17477618  0.1364513   0.16329389
   0.13449548  0.13639283  0.15024984 -0.01722025  0.3981235